In [ ]:
!pip install git+https://github.com/afnan47/cuda.git
%load_ext nvcc_plugin


  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-2ql7y1vp
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-2ql7y1vp
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=849c9017f9afec02b1f21564e3691f43d1105adb57206d5ba5e4279ba9d0c6d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-t8d95_hq/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:

%%writefile matrix_mult.cu
#include <iostream>
#include <cuda.h>
using namespace std;
#define BLOCK_SIZE 1
__global__ void gpuMM(float *A, float *B, float *C, int N) {
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
float sum = 0.f;
for (int n = 0; n < N; ++n)
sum += A[row * N + n] * B[n * N + col];
C[row * N + col] = sum;
}
int main(int argc, char *argv[]) {
int N;
// Get matrix size from user
cout << "Enter size of matrix (N): ";
cin >> N;
if (N % BLOCK_SIZE != 0) {
cerr << "Matrix size must be a multiple of BLOCK_SIZE." << endl;
return 1;
}
cout << "\nExecuting Matrix Multiplication" << endl;
cout << "Matrix size: " << N << "x" << N << endl;
// Allocate memory for matrices on the host
float *hA, *hB, *hC;
hA = new float[N * N];
hB = new float[N * N];
hC = new float[N * N];
// Read matrices from user
cout << "Enter elements of matrix A (" << N << "x" << N << "):" << endl;
for (int i = 0; i < N * N; ++i)
cin >> hA[i];
cout << "Enter elements of matrix B (" << N << "x" << N << "):" << endl;
for (int i = 0; i < N * N; ++i)
cin >> hB[i];
// Allocate memory for matrices on the device
int size = N * N * sizeof(float);
float *dA, *dB, *dC;
cudaMalloc(&dA, size);
cudaMalloc(&dB, size);
cudaMalloc(&dC, size);
// Copy matrices from the host to the device
cudaMemcpy(dA, hA, size, cudaMemcpyHostToDevice);
cudaMemcpy(dB, hB, size, cudaMemcpyHostToDevice);
dim3 threadBlock(BLOCK_SIZE, BLOCK_SIZE);
dim3 grid(N / BLOCK_SIZE, N / BLOCK_SIZE);
// Execute the matrix multiplication kernel
gpuMM<<<grid, threadBlock>>>(dA, dB, dC, N);
// Copy the result matrix from the device to the host
cudaMemcpy(hC, dC, size, cudaMemcpyDeviceToHost);
// Display the result matrix
cout << "\nResultant matrix:\n";
for (int row = 0; row < N; row++) {
for (int col = 0; col < N; col++) {
cout << hC[row * N + col] << " ";
}
cout << endl;
}
// Free device memory
cudaFree(dA);
cudaFree(dB);
cudaFree(dC);
// Free host memory
delete[] hA;
delete[] hB;
delete[] hC;
cout << "Finished." << endl;
return 0;
}


Writing matrix_mult.cu


In [1]:

!nvcc matrix_mult.cu -o matrix_mult
!./matrix_mult


cc1plus: fatal error: matrix_mult.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./matrix_mult: No such file or directory
